In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
import matplotlib
import numpy
from math import log, exp, lgamma, sqrt
import scipy.stats
from pylab import *

LOG_FACTORIAL_CACHE = [lgamma(n + 1) for n in range(1000)]
def log_factorial(n):
    return LOG_FACTORIAL_CACHE[n]

ALT_MINOR = 0
REF_MINOR = 1
OUTLIER = 2
POSSIBLE_HIDDEN_VALUES = [ALT_MINOR, REF_MINOR, OUTLIER]

In [5]:
#given unnormalized log-probabilities[c*log p1, c*log p2, c*log p3 . . . c*log pN] return normalized
## probabilities [p1, p2. ..pN] in an underflow-aware way
def probs_from_log_probs(log_probs):
    M = max(log_probs)
    unnormalized_probs = [exp(p - M) for p in log_probs]
    normalizer = sum(unnormalized_probs)
    return [p / normalizer for p in unnormalized_probs]

In [6]:
#An auxiliary function (in Java, a private static) that is the log of the factor obtained from marginalizing
#out a single bias ratio lambda
def phi(f, alpha, beta, a, r):
    if f == 1: #if f == 1 the conditional is exactly gamma already and can be marginalized immediately
        rho = alpha + r
        tau = beta
        log_c = 0
    else:
        n = a + r
        w = (1-f)*(a - alpha + 1) + beta*f #intermediate for calculating the mode of the conditional
        mode = (sqrt(w**2 + 4*beta*f*(1-f)*(alpha + r -1)) - w)/(2*beta*(1-f))
        curvature = (alpha + r -1)/(mode**2) - n*(1-f)**2/(f + (1-f)*mode)**2

        #match the mode, curvature, and normalization to an effective gamma conditional
        tau = mode*curvature
        rho = mode*tau + 1
        log_c = (r + alpha - rho)*log(mode) + (tau - beta)*mode - n * log(f + (1-f)*mode)
    
    return log_c + lgamma(rho) - rho*log(tau)
            

In [7]:
#combine the marginalization factor with f- and pi-dependent factors to get the log-likelihood
#contribution from a single site.  In Java this could be  member function but in Python
#all the self. gets messy and tedious
def log_likelihood_original_parameters(f, alpha, beta, a, r, pi):
        log_alt_minor_marginalization = phi(f, alpha, beta, a, r)
        log_ref_minor_marginalization = phi(1-f, alpha, beta, a, r)
        log_outlier_marginalization = lgamma(alpha) - alpha * log(beta)
        
        log_alt_minor_term = log_alt_minor_marginalization + log(1-pi) + a * log(f) + r * log(1-f)
        log_ref_minor_term = log_ref_minor_marginalization + log(1-pi) + r * log(f) + a * log(1-f)
        log_outlier_term = log_outlier_marginalization + log(2*pi) + log_factorial(a) + log_factorial(r) - log_factorial(a+r+1)
        return log_sum_log(log_alt_minor_term, log_ref_minor_term, log_outlier_term) + alpha*log(beta) - lgamma(alpha)
    
def log_likelihood(f, mu, beta, a, r, pi):
    return log_likelihood_original_parameters(f, mu*beta, beta, a, r, pi)

In [8]:
#overflow- and underflow-savvy log(exp(a)+exp(b)+exp(c))
#log(exp(a)+exp(b)+exp(c)) = log[exp(M)*(exp(a-M)+exp(b-M)+exp(c-M))] where  M = max(a,b,c)
#                          = M + log[(exp(a-M)+exp(b-M)+exp(c-M))]
def log_sum_log(a,b,c):
    M = max(a,b,c)
    return M + log(exp(a-M)+exp(b-M)+exp(c-M))

In [9]:
class AlleleFractionModel:
    
    def __init__(self, alt_counts, ref_counts, segment_lengths):
        self.a = alt_counts
        self.r = ref_counts
        self.N = len(alt_counts)
        self.S = len(segment_lengths)
        
        segment_ends = numpy.cumsum(segment_lengths)
        segment_starts = [0]
        segment_starts.extend(segment_ends[:-1])
        
        self.het_to_segment = [0 for n in range(self.N)]
        self.segment_ranges = []
        for s, start_end in enumerate(zip(segment_starts, segment_ends)):
            start, end = start_end
            for j in range(start, end):
                self.het_to_segment[j] = s
            self.segment_ranges.append(range(start, end))
        
        self.pi = 0.01 
        self.mu, self.beta = 1, 10
        
        #initialize f naively
        self.f = [0.5 for s in range(self.S)]
        for s in range(self.S):
            total_minor, total_major = 0, 0
            for h in self.segment_ranges[s]:
                total_minor += min(self.a[h], self.r[h])
                total_major += max(self.a[h], self.r[h])
            self.f[s] = min(1, (total_minor + 1) / (total_major + total_minor + 2))
        
        #set up the adaptive metropolis samplers
        self.f_sampler = [AdaptiveMetropolisSampler(0.01, 0, 1) for s in range(self.S)]
        self.mu_sampler = AdaptiveMetropolisSampler(0.01, 0)
        self.beta_sampler = AdaptiveMetropolisSampler(10,0,1000)
        self.pi_sampler = AdaptiveMetropolisSampler(0.1, 0, 0.1)
        
    def get_f(self, h):
        return self.f[self.het_to_segment[h]]
        
    def update_pi(self):
        self.pi = self.pi_sampler.sample(self.pi, lambda pi: self.log_conditional_on_pi(pi))
    
    def update_f(self, s):
        self.f[s] = self.f_sampler[s].sample(self.f[s], lambda f: self.log_conditional_on_f(f, s))
    
    def update_mu(self):
        self.mu = self.mu_sampler.sample(self.mu, lambda mu: self.log_conditional_on_mu(mu))
    
    def update_beta(self):
        self.beta = self.beta_sampler.sample(self.beta, lambda beta: self.log_conditional_on_beta(beta))
    
    #fix everything except pi
    def log_conditional_on_pi(self, pi):
        return sum( log_likelihood(self.get_f(h), self.mu, self.beta, self.a[h], self.r[h], pi) for h in range(self.N))
    
    #fix everything except mu
    def log_conditional_on_mu(self, mu):
        return sum( log_likelihood(self.get_f(h), mu, self.beta, self.a[h], self.r[h], self.pi) for h in range(self.N))
    
    #fix everything except beta
    def log_conditional_on_beta(self, beta):
        return sum( log_likelihood(self.get_f(h), self.mu, beta, self.a[h], self.r[h], self.pi) for h in range(self.N))
    
    #fix everything except f[s]
    def log_conditional_on_f(self, f, s):
        return sum( log_likelihood(f, self.mu, self.beta, self.a[h], self.r[h], self.pi) for h in self.segment_ranges[s])  
    
    #site h's contribution to the log-likelihood
    def log_likelihood_contribution(self, h):
        log_alt_minor_marginalization = phi(self.get_f(h), self.alpha, self.beta, self.a[h], self.r[h])
        log_ref_minor_marginalization = phi(1 - self.get_f(h), self.alpha, self.beta, self.a[h], self.r[h])
        log_outlier_marginalization = lgamma(self.alpha) - self.alpha * log(self.beta)
        
        log_alt_minor_term = log_alt_minor_marginalization + log(1-self.pi)
        log_ref_minor_term = log_alt_minor_marginalization
        log_outlier_term = log_alt_minor_marginalization
            
    def mcmc_iteration(self):
        for s in range(self.S):
            self.update_f(s)
        self.update_mu()
        self.update_beta()
        self.update_pi()
        
    def report(self):
        print("Mu, beta, pi: ", self.mu, self.beta, self.pi)
        
    def mcmc(self, num_burn_in, num_iterations):
        for i in range(num_burn_in):
            print("Iteration %d out of %d burn-in iterations" % (i, num_burn_in))
            self.mcmc_iteration()
            self.report()
            
        f_samples = []
        mu_samples = []
        beta_samples = []
        pi_samples = []
        
        for i in range(num_iterations):
            print("Iteration %d out of %d iterations" % (i, num_iterations))
            self.mcmc_iteration()
            self.report()
            f_samples.append(self.f.copy())
            pi_samples.append(self.pi)
            mu_samples.append(self.mu)
            beta_samples.append(self.beta)
        
        return f_samples, pi_samples, mu_samples, beta_samples

In [10]:
class AdaptiveMetropolisSampler:
    '''
    This sampler makes symmetric Metropolis proposals x_proposed = x_current + delta_x,
    where delta_x ~ Cauchy(width). We choose a Cauchy because of its fat tails.  
    
    Idea: take the proposal as a lambda
    
    right now I assume sampling a positive quanity, hence take the absolute value of the proposal
    (this is symmetric).
    
    The width is tuned to achieve a desired acceptance rate via the Robbins-Monro algorithm.
    In order to be define a valid MCMC algorithm the amount of tuning at each step must go to zero.  
    This sampler achieves this by scaling correction at the nth iteration by a/(b+n)
    
    '''
    def __init__(self, width=0.1, lower=float("-inf"), upper=float("inf")):
        self.lower = lower
        self.upper = upper
        self.a = 20
        self.b = 20
        self.width = width
        self.optimal_acceptance_rate = 0.4
        self.n = 1
    
    ##sample given a current value and a lambda to compute the log probability
    def sample(self, x_old, log_probability):
        x_new = x_old + scipy.stats.cauchy.rvs()*self.width
        
        if x_new < self.lower or x_new > self.upper:
            acceptance_probability = 0.0
        else:
            acceptance_probability = min(1, exp(log_probability(x_new) - log_probability(x_old)))
            
        correction_factor = (acceptance_probability - self.optimal_acceptance_rate) * self.a / (self.b + self.n)
        self.width *= math.exp(correction_factor)
        self.n += 1
        
        return x_new if scipy.stats.uniform.rvs() < acceptance_probability else x_old

In [11]:
##generate inputs to MLE_solution using same generative model as our inference
def generate_data(average_hets_in_segment, num_segments, average_depth, alpha, beta, pi):
    MIN_HETS_PER_SEGMENT = 3
    a = []
    r = []
    
    start_het = [0 for s in range(num_segments)]
    end_het = [0 for s in range(num_segments)]
    segments = []
    f = []
    num_alt_minor = 0
    num_ref_minor = 0
    
    biases = []
    h = 0
    for s in range(num_segments):
        start_het[s] = h
        minor_fraction = scipy.stats.uniform.rvs(0, 0.5)
        f.append(minor_fraction)
        num_hets = MIN_HETS_PER_SEGMENT + scipy.stats.poisson.rvs(mu=average_hets_in_segment)
        for j in range(num_hets):
            segments.append(s)
            bias = scipy.stats.gamma.rvs(alpha, scale=1/beta)
            biases.append(bias)
            
            is_outlier = scipy.stats.bernoulli.rvs(pi)
            if is_outlier:
                alt_prob = scipy.stats.uniform.rvs()
            else:
                indicator = scipy.stats.bernoulli.rvs(0.5)

                if indicator == ALT_MINOR:
                    num_alt_minor += 1
                    f_alt = minor_fraction
                if indicator == REF_MINOR:
                    num_ref_minor += 1
                    f_alt = 1 - minor_fraction

                alt_prob = f_alt/(f_alt + (1-f_alt)*bias)
                
            total_count = 1 + scipy.stats.poisson.rvs(mu=average_depth)
            a.append(scipy.stats.binom.rvs(n=total_count, p = alt_prob))
            r.append(total_count - a[h])
            h+=1
            
        end_het[s] = h
        segment_lengths = [e - s for e, s in zip(end_het, start_het)]
    return a, r, segment_lengths, f, biases
    

In [12]:
def test(average_hets_in_segment, num_segments, average_depth, mu, beta, pi, num_iterations, burn_in):
    alpha = mu*beta
    a, r, segment_lengths, true_f, true_biases = generate_data(average_hets_in_segment, num_segments, average_depth, alpha, beta, pi)
    true_mean_bias = numpy.mean(true_biases)
    #print(true_f)
    model = AlleleFractionModel(a, r, segment_lengths)
    f_samples, pi_samples, mu_samples, beta_samples = model.mcmc(burn_in, num_iterations)
    
    print("actual mean bias, posterior mean bias,  standard deviation of mean bias")
    print("%8.3f %8.3f %8.3f" % ( mu, numpy.mean(mu_samples), numpy.std(mu_samples)))
    
    print("actual inverse overdispersion, posterior inverse overdispersion,  standard deviation of inverse overdispersion")
    print("%8.3f %8.3f %8.3f" % ( beta, numpy.mean(beta_samples), numpy.std(beta_samples)))
                              
    print("actual outlier robability, posterior mean outlier probability,  standard deviation of outlier probability")
    print("%8.3f %8.3f %8.3f" % ( pi, numpy.mean(pi_samples), numpy.std(pi_samples)))

    print("The true BAF, estimated BAF (mean of MCMC samples), and error bars of estimate (standard deviation of MCMC samples) are:")
    print("True      Estimate  Error_Bars")
    
    for s in range(len(true_f)):
        f = [sample[s] for sample in f_samples]
        f_minor = [x if x < 0.5 else (1 - x) for x in f]
        print("%8.3f %8.3f %8.3f" % (true_f[s], numpy.mean(f_minor), numpy.std(f_minor)))

In [ ]:
test(average_hets_in_segment=20, num_segments=500, average_depth=40, mu=1.09, beta = 120, pi=0.03, num_iterations=200, burn_in=50)